# DSCI6003 Lab 4.3 - Implementing Bernoulli Bayes

# Please do this entire lab in two seperate .py files (bernoulli.py and run_bernoulli.py). The first will contain your implementation of the algorithm. The next, when I run it, should contain the scores of your code versus sklearn's. 

## Part 1: Implement the code.
Let's continue with the algorithm coding streak! We will be implementing Bernoulli Bayes today. There is a test file to make sure your code is implemented correctly.

## Part 2: Test it against sklearn.
We are going to use a dataset containing e-mails that are labeled as either spam(1) or not spam(0). Make sure your code is scoring similarly to sklearn using <a href = https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/>this dataset</a>. 

1. Load in the dataset using Pandas (please, I'm begging you use Pandas and make sure header = None).

2. Look at the data. We want to make sure we keep the columns that correspond to whether or not a word is present. Store the columns that contain information about whether or not a word is present. 

3. Create a threshold for your X values, so that anything above this threshold is a 1 and below is 0. You need to do this because BernoulliNB works on binarized data. 

4. Get the labels (last column). 

5. Run KFold cross validation with 5 folds and see how your score compares to sklearns.

6. Print out your accuracy against sklearns. 

7. Now, use sklearn's MultinomialNB classifier on this dataset. Does Bernoulli or Naive Bayes perform better? Try using different thresholds. Print out your conclusions (i.e. print("____ algorithm preforms better"))